<a href="https://colab.research.google.com/github/Suffragette/github-slideshow/blob/master/%CE%95%CF%81%CE%B3%CE%B1%CF%83%CF%84%CE%B7%CF%81%CE%B9%CE%B1%CE%BA%CE%AE_%CE%86%CF%83%CE%BA%CE%B7%CF%83%CE%B7_%CE%A4%CE%B5%CF%87%CE%BD%CE%B9%CE%BA%CE%AD%CF%82_%CE%94%CE%B9%CE%B1%CF%87%CE%B5%CE%AF%CF%81%CE%B9%CF%83%CE%B7%CF%82_%CE%BA%CE%B1%CE%B9_%CE%95%CE%BE%CF%8C%CF%81%CF%85%CE%BE%CE%B7%CF%82_%CE%B3%CE%B9%CE%B1_%CE%94%CE%B5%CE%B4%CE%BF%CE%BC%CE%AD%CE%BD%CE%B1_%CE%9C%CE%B5%CE%B3%CE%AC%CE%BB%CE%BF%CF%85_%CE%8C%CE%B3%CE%BA%CE%BF%CF%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark==3.1.2
!pip install spark-nlp==3.3.1

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
import sparknlp

spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[4]") \
    .config("spark.driver.memory","16G") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.1") \
    .getOrCreate()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)


Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,866 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,529 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 5,731 kB in 2s

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Load the data
data = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_train.csv", inferSchema=True, header=True)
data = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_labels.csv", inferSchema=True, header=True)
data = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_test.csv", inferSchema=True, header=True)

# Show the first few rows of the DataFrame
data.show()


+---------------+--------------------+
|condition_label|    medical_abstract|
+---------------+--------------------+
|              3|Obstructive sleep...|
|              5|Neutrophil functi...|
|              5|A phase II study ...|
|              1|Flow cytometric D...|
|              4|Paraneoplastic va...|
|              1|Treatment of chil...|
|              1|Expression of maj...|
|              1|Questionable role...|
|              5|Reversibility of ...|
|              2|Current status of...|
|              5|The importance of...|
|              1|Human papillomavi...|
|              5|Gentamicin iontop...|
|              1|Repeat hepatic re...|
|              5|Evidence for intr...|
|              5|Glutamic acid and...|
|              5|A useful techniqu...|
|              5|The natural histo...|
|              3|Hereditary intern...|
|              5|Immune response o...|
+---------------+--------------------+
only showing top 20 rows



In [ ]:
print(data.columns)


['condition_label', 'medical_abstract']


In [ ]:
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer, StopWordsCleaner

# Assemble the text column into a Document type column
document_assembler = DocumentAssembler() \
    .setInputCol("medical_abstract") \
    .setOutputCol("document")

# Tokenize the document
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Normalize the tokens (remove punctuation and convert to lowercase)
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

# Remove stopwords
stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(["normalized"]) \
    .setOutputCol("clean_tokens") \
    .setCaseSensitive(False)

# Finish the preprocessing (convert the result back to a string column)
finisher = Finisher() \
    .setInputCols(["clean_tokens"]) \
    .setOutputCols(["clean_text"]) \
    .setOutputAsArray(False) \
    .setCleanAnnotations(False)

# Define the stages of the pipeline
stages = [document_assembler, tokenizer, normalizer, stopwords_cleaner, finisher]

# Create a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data)

# Transform the data
data_clean = model.transform(data)

# Show the first few rows of the cleaned data
data_clean.show()


+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          clean_text|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|Obstructive@sleep...|
|              5|Neutrophil functi...|[{document, 0, 16...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|Neutrophil@functi...|
|              5|A phase II study ...|[{document, 0, 18...|[{token, 0, 0, A,...|[{token, 0, 0, A,...|[{token, 2, 6, ph...|phase@II@study@co...|
|              1|Flow cytometric D...|[{document, 0, 19...|[{token, 0, 3, Fl...|[{token, 0, 3, Fl...|[{token, 0, 3, Fl...|Flow@cytometri

In [ ]:
from sparknlp.annotator import WordEmbeddingsModel

# Load a pre-trained Word Embeddings model
embeddings = WordEmbeddingsModel.pretrained() \
    .setInputCols(["document", "clean_tokens"]) \
    .setOutputCol("embeddings")

# Add the embeddings stage to the pipeline
stages.append(embeddings)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_clean)

# Transform the data
data_embedded = model.transform(data_clean)

# Show the first few rows of the embedded data
data_embedded.show()


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          clean_text|          embeddings|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|Obstructive@sleep...|[{word_embeddings...|
|              5|Neutrophil functi...|[{document, 0, 16...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|Neutrophil@functi...|[{word_embeddings...|
|              5|A phase II study ...|[{document, 0, 18...|[{token, 0, 0, A,..

In [ ]:
from sparknlp.annotator import BertEmbeddings

# Load a pre-trained BERT model
bert = BertEmbeddings.pretrained() \
    .setInputCols(["document", "clean_tokens"]) \
    .setOutputCol("bert_embeddings")

# Add the BERT embeddings stage to the pipeline
stages.append(bert)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_clean)

# Transform the data
data_bert = model.transform(data_clean)

# Show the first few rows of the BERT-embedded data
data_bert.show()


small_bert_L2_768 download started this may take some time.
Approximate size to download 139.6 MB
[OK!]
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          clean_text|          embeddings|     bert_embeddings|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|Obstructive@sleep...|[{word_embeddings...|[{word_embeddings...|
|              5|Neutrophil functi...|[{document, 0, 16...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|Neutrophil@functi...|[{word_em

In [ ]:
from sparknlp.annotator import SentenceEmbeddings

# Create sentence vectors by summing the word vectors
sentence_embeddings_sum = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings_sum") \
    .setPoolingStrategy("SUM")

# Create sentence vectors by averaging the word vectors
sentence_embeddings_avg = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings_avg") \
    .setPoolingStrategy("AVERAGE")

# Add the sentence embeddings stages to the pipeline
stages.append(sentence_embeddings_sum)
stages.append(sentence_embeddings_avg)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_embedded)

# Transform the data
data_sentence = model.transform(data_embedded)

# Show the first few rows of the sentence-embedded data
data_sentence.show()


+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          clean_text|          embeddings|     bert_embeddings|sentence_embeddings_sum|sentence_embeddings_avg|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|Obstructive@sleep...|[{word_embeddings...|[{word_embeddings...|   [{sentence_embedd...|   [{sentence_embedd...|
|              5|Neutrophil functi...|[{document, 0, 16...|[{tok

In [ ]:
print(data.columns)


['condition_label', 'medical_abstract']


In [ ]:
for stage in stages:
    print(stage)


DocumentAssembler_f10219acd759
Tokenizer_6d708740dada
Normalizer_f5866a41a23d
StopWordsCleaner_7e4c74e9f04f
Finisher_ac66ee570be8
WORD_EMBEDDINGS_MODEL_48cffc8b9a76
SentenceEmbeddings_5074f733abad
SentenceEmbeddings_d434c0f95fd1
VectorAssembler_4a9088914f21


In [ ]:
from sparknlp.annotator import BertEmbeddings

# Load a pre-trained BERT model
bert = BertEmbeddings.pretrained() \
    .setInputCols(["document", "clean_tokens"]) \
    .setOutputCol("bert_embeddings")

# Add the BERT embeddings stage to the pipeline
stages.append(bert)


small_bert_L2_768 download started this may take some time.
Approximate size to download 139.6 MB
[OK!]


In [ ]:
data_sentence.printSchema()


In [ ]:
# Convert the BERT embeddings into sentence embeddings
sentence_embeddings = SentenceEmbeddings() \
    .setInputCols(["document", "bert_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType

# Define a UDF that extracts the embeddings from the struct
def extract_embeddings(embedding_struct):
    return [float(x) for x in embedding_struct.embeddings]

# Register the UDF
udf_extract_embeddings = udf(extract_embeddings, ArrayType(FloatType()))

# Apply the UDF to the sentence embeddings
data_embeddings = data_sentence.withColumn("sentence_vector", udf_extract_embeddings("sentence_embeddings_avg"))

# Print the schema to check the result
data_embeddings.printSchema()


In [ ]:
from pyspark.ml.feature import VectorAssembler


In [ ]:
assembler = VectorAssembler(
    inputCols=["sentence_embeddings"],
    outputCol="features"
)


In [ ]:
data_features = assembler.transform(data_sentence)


IllegalArgumentException: ignored

In [ ]:
# Add the sentence embeddings stage to the pipeline
stages.append(sentence_embeddings)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_clean)

# Transform the data
data_sentence = model.transform(data_clean)


IllegalArgumentException: ignored

In [ ]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import StringType

# Define a UDF to convert the sentence embeddings into a dense vector
def to_vector(embeddings):
    return Vectors.dense(embeddings[0].embeddings)

# Register the UDF
udf_to_vector = udf(to_vector, StringType())

# Apply the UDF to the sentence embeddings
data_finished = data_sentence.withColumn("sentence_vector", udf_to_vector("sentence_embeddings"))

# Assemble the sentence vector into a feature vector
assembler = VectorAssembler() \
    .setInputCols(["sentence_vector"]) \
    .setOutputCol("features")

# Transform the data
data_features = assembler.transform(data_finished)

# Show the first few rows of the data with feature vectors
data_features.show()


AnalysisException: ignored

In [ ]:
from sparknlp.base import EmbeddingsFinisher

# Convert the sentence embeddings into a format that can be used by the VectorAssembler
embeddings_finisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCols(["sentence_vector"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

# Add the embeddings finisher stage to the pipeline
stages.append(embeddings_finisher)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_clean)

# Transform the data
data_finished = model.transform(data_clean)

# Assemble the sentence vector into a feature vector
assembler = VectorAssembler() \
    .setInputCols(["sentence_vector"]) \
    .setOutputCol("features")

# Transform the data
data_features = assembler.transform(data_finished)

# Show the first few rows of the data with feature vectors
data_features.show()


IllegalArgumentException: ignored

In [ ]:
from sparknlp.annotator import SentenceEncoder

# Convert the sentence embeddings into a format that can be used by the VectorAssembler
sentence_encoder = SentenceEncoder() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("sentence_vector")

# Add the sentence encoder stage to the pipeline
stages.append(sentence_encoder)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_clean)

# Transform the data
data_encoded = model.transform(data_clean)

# Assemble the sentence vector into a feature vector
assembler = VectorAssembler() \
    .setInputCols(["sentence_vector"]) \
    .setOutputCol("features")

# Transform the data
data_features = assembler.transform(data_encoded)

# Show the first few rows of the data with feature vectors
data_features.show()


ImportError: ignored

In [ ]:
from sparknlp.base import EmbeddingsFinisher

# Finish the embeddings
embeddings_finisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCols(["finished_sentence_embeddings"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

# Add the embeddings finisher stage to the pipeline
stages.append(embeddings_finisher)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_clean)

# Transform the data
data_finished = model.transform(data_clean)

# Assemble the finished sentence embeddings into a feature vector
assembler = VectorAssembler() \
    .setInputCols(["finished_sentence_embeddings"]) \
    .setOutputCol("features")

# Transform the data
data_features = assembler.transform(data_finished)

# Show the first few rows of the data with feature vectors
data_features.show()


IllegalArgumentException: ignored

In [ ]:
from sparknlp.annotator import BertSentenceEmbeddings

# Convert the BERT embeddings into sentence embeddings
sentence_embeddings = BertSentenceEmbeddings() \
    .setInputCols(["document", "bert_embeddings"]) \
    .setOutputCol("sentence_embeddings")

# Add the sentence embeddings stage to the pipeline
stages.append(sentence_embeddings)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_clean)

# Transform the data
data_sentence = model.transform(data_clean)

# Assemble the sentence embeddings into a feature vector
assembler = VectorAssembler() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("features")

# Transform the data
data_features = assembler.transform(data_sentence)

# Show the first few rows of the data with feature vectors
data_features.show()


IllegalArgumentException: ignored

In [ ]:
from pyspark.ml.feature import VectorAssembler
from sparknlp.annotator import SentenceEmbeddings

# Convert the BERT embeddings into sentence embeddings
sentence_embeddings = SentenceEmbeddings() \
    .setInputCols(["document", "bert_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

# Add the sentence embeddings stage to the pipeline
stages.append(sentence_embeddings)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_clean)

# Transform the data
data_sentence = model.transform(data_clean)

# Assemble the sentence embeddings into a feature vector
assembler = VectorAssembler() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("features")

# Transform the data
data_features = assembler.transform(data_sentence)

# Show the first few rows of the data with feature vectors
data_features.show()


IllegalArgumentException: ignored

In [ ]:
from sparknlp.base import SentenceEmbeddings, EmbeddingsFinisher

# Convert the BERT embeddings into sentence embeddings
sentence_embeddings = SentenceEmbeddings() \
    .setInputCols(["document", "bert_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

# Finish the embeddings (convert the result to a usable format)
embeddings_finisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCols(["finished_embeddings"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

# Add the sentence embeddings and embeddings finisher stages to the pipeline
stages.append(sentence_embeddings)
stages.append(embeddings_finisher)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_clean)

# Transform the data
data_finished = model.transform(data_clean)

# Show the first few rows of the finished data
data_finished.show()


ImportError: ignored

In [ ]:
from sparknlp.base import EmbeddingsFinisher

# Finish the embeddings (convert the result to a usable format)
embeddings_finisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings_avg"]) \
    .setOutputCols(["finished_embeddings"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

# Add the embeddings finisher stage to the pipeline
stages.append(embeddings_finisher)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_clean)

# Transform the data
data_finished = model.transform(data_clean)

# Show the first few rows of the finished data
data_finished.show()


IllegalArgumentException: ignored

In [ ]:
# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_clean)

# Transform the data
data_bert = model.transform(data_clean)

# Show the first few rows of the BERT-embedded data
data_bert.show()


IllegalArgumentException: ignored

In [ ]:
# Remove the duplicate BERT embeddings stage
stages.remove(bert)

# Now you can continue with your pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the training data
model = pipeline.fit(train_data)

# Transform the training data
train_data = model.transform(train_data)



ValueError: ignored

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Convert the array of floats into a dense vector
assembler = VectorAssembler(inputCols=["sentence_embeddings_avg"], outputCol="features")

# Add the assembler stage to the pipeline
stages.append(assembler)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the training data
model = pipeline.fit(train_data)

# Transform the training data
train_data = model.transform(train_data)


AnalysisException: ignored

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Convert target labels into numerical form
indexer = StringIndexer(inputCol="condition_label", outputCol="label")

# Convert sentence embeddings into features
assembler = VectorAssembler(inputCols=["sentence_embeddings_avg"], outputCol="features")

# Define the classifier
classifier = RandomForestClassifier(labelCol="label", featuresCol="features")

# Define the pipeline
pipeline = Pipeline(stages=[indexer, assembler, classifier])

# Split the data into training and test sets
train_data, test_data = data_sentence.randomSplit([0.8, 0.2])

# Fit the pipeline to the training data
model = pipeline.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g" % accuracy)


IllegalArgumentException: ignored

In [ ]:
# Create a pipeline for the BERT embeddings
bert_pipeline = Pipeline().setStages([document_assembler, tokenizer, normalizer, stopwords_cleaner, bert])

# Fit the pipeline to the data
bert_model = bert_pipeline.fit(data_clean)

# Transform the data
data_bert = bert_model.transform(data_clean)

# Create a pipeline for the sentence embeddings
sentence_pipeline = Pipeline().setStages([sentence_embeddings_sum, sentence_embeddings_avg])

# Fit the pipeline to the data
sentence_model = sentence_pipeline.fit(data_bert)

# Transform the data
data_sentence = sentence_model.transform(data_bert)

# Show the first few rows of the sentence-embedded data
data_sentence.show()


IllegalArgumentException: ignored

In [ ]:
from sparknlp.annotator import BertSentenceEmbeddings

# Load a pre-trained BERT model
bert_sentence = BertSentenceEmbeddings.pretrained('sent_small_bert_L2_768') \
    .setInputCols(["document"]) \
    .setOutputCol("bert_sentence_embeddings")

# Add the BERT embeddings stage to the pipeline
stages = [document_assembler, tokenizer, normalizer, stopwords_cleaner, finisher, bert_sentence]

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data)

# Transform the data
data_bert = model.transform(data)

# Show the first few rows of the BERT-embedded data
data_bert.show()


sent_small_bert_L2_768 download started this may take some time.
Approximate size to download 139.6 MB
[OK!]
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          clean_text|bert_sentence_embeddings|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|Obstructive@sleep...|    [{sentence_embedd...|
|              5|Neutrophil functi...|[{document, 0, 16...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|Neutrophil@functi...|    [{sentence_embedd...|
|              5|A phase II study ...|[{docume

In [ ]:
print(data.columns)


['condition_label', 'medical_abstract']


In [ ]:
from sparknlp.annotator import SentenceEmbeddings

# Create sentence vectors by summing the word vectors
sentence_embeddings_sum = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings_sum") \
    .setPoolingStrategy("SUM")

# Create sentence vectors by averaging the word vectors
sentence_embeddings_avg = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings_avg") \
    .setPoolingStrategy("AVERAGE")

# Add the sentence embeddings stages to the pipeline
stages.append(sentence_embeddings_sum)
stages.append(sentence_embeddings_avg)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_embedded)

# Transform the data
data_sentence = model.transform(data_embedded)

# Show the first few rows of the sentence-embedded data
data_sentence.show()


+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+-----------------------+-----------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          clean_text|          embeddings|bert_sentence_embeddings|sentence_embeddings_sum|sentence_embeddings_avg|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+-----------------------+-----------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|Obstructive@sleep...|[{word_embeddings...|    [{sentence_embedd...|   [{sentence_embedd...|   [{sentence_embedd...|
|              5|Neutrophil functi...|[{document

In [ ]:
print(data.columns)


['condition_label', 'medical_abstract']


In [ ]:
data_embedded.show()

+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          clean_text|          embeddings|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|Obstructive@sleep...|[{word_embeddings...|
|              5|Neutrophil functi...|[{document, 0, 16...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|Neutrophil@functi...|[{word_embeddings...|
|              5|A phase II study ...|[{document, 0, 18...|[{token, 0, 0, A,...|[{token, 0, 0, A,...|[{token, 2, 6, ph...|phase@II@study@co...|[{word_embeddings...|
|         

In [ ]:
from sparknlp.annotator import SentenceEmbeddings

# Create sentence vectors by summing the word vectors
sentence_embeddings_sum = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings_sum") \
    .setPoolingStrategy("SUM")

# Create sentence vectors by averaging the word vectors
sentence_embeddings_avg = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings_avg") \
    .setPoolingStrategy("AVERAGE")

# Add the sentence embeddings stages to the pipeline
stages.append(sentence_embeddings_sum)
stages.append(sentence_embeddings_avg)

# Create a new pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data_embedded)

# Transform the data
data_sentence = model.transform(data_embedded)

# Show the first few rows of the sentence-embedded data
data_sentence.show()


+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+-----------------------+-----------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          clean_text|          embeddings|bert_sentence_embeddings|sentence_embeddings_sum|sentence_embeddings_avg|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+-----------------------+-----------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|Obstructive@sleep...|[{word_embeddings...|    [{sentence_embedd...|   [{sentence_embedd...|   [{sentence_embedd...|
|              5|Neutrophil functi...|[{document

In [ ]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

# Define a UDF to convert array type to vector type
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

# Convert the 'sentence_embeddings_sum' column from array type to vector type
data_sentence = data_sentence.select(
    "*",
    list_to_vector_udf(data_sentence["sentence_embeddings_sum"]).alias("sentence_embeddings_vector")
)

# Now you can use 'sentence_embeddings_vector' as your features column


In [ ]:
from sparknlp.annotator import BertSentenceEmbeddings

# Define the sentence embeddings transformer
sentence_embeddings = BertSentenceEmbeddings.pretrained("sent_small_bert_L2_768", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("new_bert_sentence_embeddings")

# Transform the data
data_sentence = sentence_embeddings.transform(data_sentence)

# Use the EmbeddingsFinisher to flatten the embeddings
finisher = EmbeddingsFinisher() \
    .setInputCols(["new_bert_sentence_embeddings"]) \
    .setOutputCols(["sentence_embeddings_vector"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

data_sentence = finisher.transform(data_sentence)

# Now you can use 'sentence_embeddings_vector' as your features column



sent_small_bert_L2_768 download started this may take some time.
Approximate size to download 139.6 MB
[OK!]


In [ ]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

# Define a UDF to convert array type to vector type
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

# Convert the 'sentence_embeddings_vector' column
data_sentence = data_sentence.withColumn("sentence_embeddings_vector", list_to_vector_udf("sentence_embeddings_vector"))

# Now you can use 'sentence_embeddings_vector' as your features column


In [ ]:
if 'bert_sentence_embeddings' in data_sentence.columns:
    data_sentence = data_sentence.drop('bert_sentence_embeddings')


In [ ]:
# Define the sentence embeddings transformer
sentence_embeddings = BertSentenceEmbeddings.pretrained("sent_small_bert_L2_768", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("bert_sentence_embeddings")

# Transform the data
data_sentence = sentence_embeddings.transform(data_sentence)

# Use the EmbeddingsFinisher to flatten the embeddings
finisher = EmbeddingsFinisher() \
    .setInputCols(["bert_sentence_embeddings"]) \
    .setOutputCols(["sentence_embeddings_vector"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

data_sentence = finisher.transform(data_sentence)


sent_small_bert_L2_768 download started this may take some time.
Approximate size to download 139.6 MB
[OK!]


In [ ]:
# Split the data into training and test sets
train_data, test_data = data_sentence.randomSplit([0.8, 0.2])


In [ ]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

# Define a UDF to convert array type to vector type
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

# Extract the 'embeddings' field from the 'bert_sentence_embeddings' column and convert it to vector type
data_sentence = data_sentence.withColumn("features", list_to_vector_udf(data_sentence["bert_sentence_embeddings"].getItem("embeddings")))

# Now you can use 'features' as your features column


In [ ]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

# Define a UDF to convert array type to vector type
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

# Extract the 'embeddings' field from the 'bert_sentence_embeddings' column and convert it to vector type
data_sentence = data_sentence.withColumn("features", list_to_vector_udf(data_sentence["bert_sentence_embeddings"].embeddings))

# Now you can use 'features' as your features column


In [ ]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

# Define a UDF to convert array type to vector type
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

# Extract the 'embeddings' field from the 'bert_sentence_embeddings' column and convert it to vector type
data_sentence = data_sentence.withColumn("features", list_to_vector_udf(data_sentence["bert_sentence_embeddings"].embeddings))

# Now you can use 'features' as your features column


In [ ]:
from sparknlp.base import EmbeddingsFinisher

finisher = EmbeddingsFinisher() \
    .setInputCols(["bert_sentence_embeddings"]) \
    .setOutputCols(["finished_embeddings"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

data_sentence = finisher.transform(data_sentence)


In [ ]:
from sparknlp.annotator import BertEmbeddings

# Define BertEmbeddings
bert_embeddings = BertEmbeddings.pretrained('bert_base_cased', 'en') \
    .setInputCols(["document",'token'])\
    .setOutputCol("bert_embeddings")


bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [ ]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

# Define a UDF to convert array type to vector type
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

# Convert the 'sentence_embeddings_vector' column from array type to vector type
data_sentence = data_sentence.withColumn("sentence_embeddings_vector", list_to_vector_udf("sentence_embeddings_vector"))

# Now you can use 'sentence_embeddings_vector' as your features column


In [ ]:
from sparknlp.annotator import BertSentenceEmbeddings

# Define the sentence embeddings transformer
sentence_embeddings = BertSentenceEmbeddings.pretrained("sent_small_bert_L2_768", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("bert_sentence_embeddings")

# Transform the data
data_sentence = sentence_embeddings.transform(data_sentence)

# Use the EmbeddingsFinisher to flatten the embeddings
finisher = EmbeddingsFinisher() \
    .setInputCols(["bert_sentence_embeddings"]) \
    .setOutputCols(["sentence_embeddings_vector"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

data_sentence = finisher.transform(data_sentence)

# Now you can use 'sentence_embeddings_vector' as your features column


sent_small_bert_L2_768 download started this may take some time.
Approximate size to download 139.6 MB
[OK!]


AnalysisException: ignored

In [ ]:
from pyspark.ml.classification import LinearSVC

# Split the data into training and test sets
train_data, test_data = data_sentence.randomSplit([0.8, 0.2])

# Create an SVM classifier
svm = LinearSVC() \
    .setLabelCol("condition_label") \
    .setFeaturesCol("sentence_embeddings_vector")

# Train the SVM classifier
svm_model = svm.fit(train_data)

# Make predictions on the test data
predictions = svm_model.transform(test_data)

# Show the first few rows of the predictions
predictions.show()


PythonException: ignored

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Define the assembler
assembler = VectorAssembler(
    inputCols=["sentence_embeddings_vector"],
    outputCol="features")

# Transform the data
data_sentence = assembler.transform(data_sentence)

# Now you can use 'features' as your features column


IllegalArgumentException: ignored

In [ ]:
# Define a UDF to convert array type to vector type
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

# Convert the 'finished_embeddings' column to vector type
data_sentence = data_sentence.withColumn("features", list_to_vector_udf(data_sentence["finished_embeddings"]))

# Now you can use 'features' as your features column
svm = LinearSVC() \
    .setLabelCol("condition_label") \
    .setFeaturesCol("features")

# Train the SVM classifier
svm_model = svm.fit(train_data)


IllegalArgumentException: ignored

In [ ]:
# Now you can use 'features' as your features column
svm = LinearSVC() \
    .setLabelCol("condition_label") \
    .setFeaturesCol("features")

# Train the SVM classifier
svm_model = svm.fit(train_data)


IllegalArgumentException: ignored

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create a Decision Tree classifier
dt = DecisionTreeClassifier() \
    .setLabelCol("condition_label") \
    .setFeaturesCol("finished_embeddings")

# Train the Decision Tree classifier
dt_model = dt.fit(train_data)

# Make predictions on the test data
predictions = dt_model.transform(test_data)


IllegalArgumentException: ignored

In [ ]:
# Define the pipeline
pipeline = Pipeline(
    stages=[
        document_assembler,
        tokenizer,
        normalizer,
        bert_embeddings,
        SentenceEmbeddings() \
            .setInputCols(["document", "bert_embeddings"]) \
            .setOutputCol("sentence_embeddings") \
            .setPoolingStrategy("AVERAGE"),
        finisher
    ])

# Fit and transform the data
model = pipeline.fit(data)
result = model.transform(data)

# Now you can use 'finished_embeddings' as your features column
svm = LinearSVC() \
    .setLabelCol("condition_label") \
    .setFeaturesCol("finished_embeddings")

# Split the data into train and test sets
train_data, test_data = result.randomSplit([0.7, 0.3], seed = 100)

# Train the SVM classifier
svm_model = svm.fit(train_data)

# Make predictions on the test data
predictions = svm_model.transform(test_data)


IllegalArgumentException: ignored

In [ ]:
from sparknlp.base import LightPipeline

# Define the pipeline
pipeline = Pipeline(
    stages=[
        document_assembler,
        tokenizer,
        normalizer,
        bert_embeddings,
        sentence_embeddings,
        finisher
    ])

# Fit and transform the data
model = pipeline.fit(data)
result = model.transform(data)

# Now you can use 'finished_embeddings' as your features column
svm = LinearSVC() \
    .setLabelCol("condition_label") \
    .setFeaturesCol("finished_embeddings")

# Split the data into train and test sets
train_data, test_data = result.randomSplit([0.7, 0.3], seed = 100)

# Train the SVM classifier
svm_model = svm.fit(train_data)

# Make predictions on the test data
predictions = svm_model.transform(test_data)


IllegalArgumentException: ignored

In [ ]:
from sparknlp.base import EmbeddingsFinisher

# Initialize the EmbeddingsFinisher
finisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCols(["finished_embeddings"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

# Add the finisher to the pipeline
pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    normalizer,
    bert_embeddings,
    sentence_embeddings,
    finisher
])

# Fit and transform the data
model = pipeline.fit(data)
result = model.transform(data)

# Now you can use 'finished_embeddings' as your features column
svm = LinearSVC() \
    .setLabelCol("condition_label") \
    .setFeaturesCol("finished_embeddings")

# Train the SVM classifier
svm_model = svm.fit(result)


IllegalArgumentException: ignored